In [1]:
%pip install --upgrade DBcm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\mralo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import DBcm
import os
import swim_utils
import datetime

config = {
    "user": "swimuser",
    "password": "swimPasswd",
    "database": "SwimClub",
    "host": "localhost",
    "port": 3307
}

In [3]:
files = os.listdir(swim_utils.FOLDER)
files.remove('.DS_Store')


In [4]:
swimertable = []
swimertable_query = """SELECT name, age FROM swimmers"""
with DBcm.UseDatabase(config) as db:
    db.execute(swimertable_query)
    swimertable = db.fetchall()

In [5]:
existing_names = []
for row in swimertable:
    if row not in existing_names:
        existing_names.append(row)

In [6]:
swimmers_data1 = []
for filename in files:
    name = swim_utils.get_swimmers_data(filename)[0]
    age = swim_utils.get_swimmers_data(filename)[1]

    if (name, age, name, age) not in swimmers_data1:
        swimmers_data1.append((name,age,name,age))

In [7]:
SQL = """    
    INSERT INTO swimmers
    (name, age)
    select %s, %s
    where not exists (select * from swimmers where name = %s and age = %s);
"""

with DBcm.UseDatabase(config) as db:
    for i in range(len(swimmers_data1)):
        db.execute(SQL, swimmers_data1[i])

In [8]:
eventstable = []
eventstable_query = """SELECT distance, stroke FROM events"""
with DBcm.UseDatabase(config) as db:
    db.execute(eventstable_query)
    eventstable = db.fetchall()

In [9]:
existing_stroke = []
for row in eventstable:
    if row not in existing_stroke:
        existing_stroke.append(row)


In [10]:
events_data1 = []
for filename in files:
    distance = swim_utils.get_swimmers_data(filename)[2]
    stroke = swim_utils.get_swimmers_data(filename)[3]

    if (distance, stroke, distance, stroke) not in events_data1:
        events_data1.append((distance, stroke, distance, stroke))

In [11]:
SQL = """    
    INSERT INTO events
    (distance, stroke)
    select %s, %s
    where not exists (select * from events where distance = %s and stroke = %s);
"""

with DBcm.UseDatabase(config) as db:
    for i in range(len(events_data1)):
        db.execute(SQL, events_data1[i])

In [12]:
dateTime = datetime.datetime.now()
all_Data = []
for file in files:
    swimmer_data = swim_utils.get_swimmers_data(file)
    
    # Unpack swimmer_data for readability
    name, age, distance, stroke, times, converts, average = swimmer_data
    
    # Check if a tuple of data (except times) is already in all_Data
    if (name, age, distance, stroke) not in all_Data:
        all_Data.append(swimmer_data)



In [13]:
with DBcm.UseDatabase(config) as db:
    for data in all_Data:
        # Extracting individual data
        name, age, distance, stroke, times, converts, average = data

        # Fetching swimmer and event IDs based on names, age, distance, and stroke
        query_swimmer_id = "SELECT id FROM swimmers WHERE name = %s AND age = %s;"
        query_event_id = "SELECT id FROM events WHERE distance = %s AND stroke = %s;"

        # Executing queries to get swimmer and event IDs
        db.execute(query_swimmer_id, (name, age))
        swimmer_id = db.fetchone()
        # Consume the result
        _ = db.fetchall()

        db.execute(query_event_id, (distance, stroke))
        event_id = db.fetchone()
        # Consume the result
        _ = db.fetchall()

        # Inserting data into the times table if the combination doesn't exist
        insert_query = """
            INSERT INTO times (swimmer_id, event_id, time, ts)
            SELECT %s, %s, %s, %s
            WHERE NOT EXISTS (
                SELECT * FROM times WHERE swimmer_id = %s AND event_id = %s AND time = %s
            );
        """
        for time in converts:
            db.execute(insert_query, (swimmer_id[0], event_id[0], swim_utils.build_time_string(time), dateTime, swimmer_id[0], event_id[0], swim_utils.build_time_string(time)))


In [14]:
import DBcm
import os
import swim_utils

config = {
    "user": "swimuser",
    "password": "pass",
    "database": "SwimClub",
    "host": "localhost",
    "port": 3307
}


files = os.listdir(swim_utils.FOLDER)

files.remove('.DS_Store')


eventstable = []
eventstable_query = """SELECT distance, stroke FROM events"""
with DBcm.UseDatabase(config) as db:
    db.execute(eventstable_query)
    eventstable = db.fetchall()

existing_stroke = []
for row in eventstable:
    if row not in existing_stroke:
        existing_stroke.append(row)

print (existing_stroke)
print (len(existing_stroke))

events_data1 = []
for filename in files:
    distance = swim_utils.get_swimmers_data(filename)[2]
    stroke = swim_utils.get_swimmers_data(filename)[3]

    if (distance, stroke, distance, stroke) not in events_data1:
        events_data1.append((distance, stroke, distance, stroke))


SQL = """    
    INSERT INTO events
    (distance, stroke)
    select %s, %s
    where not exists (select * from events where distance = %s and stroke = %s);
"""

with DBcm.UseDatabase(config) as db:
    for i in range(len(events_data1)):
        db.execute(SQL, events_data1[i])

[('100m', 'Back'), ('100m', 'Breast'), ('50m', 'Back'), ('50m', 'Breast'), ('50m', 'Free'), ('100m', 'Free'), ('200m', 'Back'), ('100m', 'Fly'), ('50m', 'Fly'), ('200m', 'IM'), ('200m', 'Breast'), ('200m', 'Free'), ('400m', 'Free'), ('100m', 'IM')]
14


In [15]:
import datetime

dateTime = datetime.datetime.now()
all_Data = []
for file in files:
    swimmer_data = swim_utils.get_swimmers_data(file)
    
    # Unpack swimmer_data for readability
    name, age, distance, stroke, times, converts, average = swimmer_data
    
    # Check if a tuple of data (except times) is already in all_Data
    if (name, age, distance, stroke) not in all_Data:
        all_Data.append(swimmer_data)

# Connecting to the database and performing operations
with DBcm.UseDatabase(config) as db:
    for data in all_Data:
        # Extracting individual data
        name, age, distance, stroke, times, converts, average = data
        
        # Fetching swimmer and event IDs based on names, age, distance, and stroke
        query_swimmer_id = "SELECT id FROM swimmers WHERE name = %s AND age = %s;"
        query_event_id = "SELECT id FROM events WHERE distance = %s AND stroke = %s;"
        
        # Executing queries to get swimmer and event IDs
    print("Querying for swimmer ID with:", name, age)
    db.execute(query_swimmer_id, (name, age))
    swimmer_id_result = db.fetchone()

# Print the result to understand what's returned
    print("Swimmer ID Result:", swimmer_id_result)

# Extract the swimmer ID if the result is not None
    swimmer_id = swimmer_id_result[0] if swimmer_id_result else None

# Print the swimmer ID
    print("Swimmer ID:", swimmer_id)

# Consume any remaining results
    db.fetchall()
    print("Querying for event ID with:", distance, stroke)
    db.execute(query_event_id, (distance, stroke))
    event_id_result = db.fetchone()

# Print the result to understand what's returned
    print("Event ID Result:", event_id_result)

# Extract the event ID if the result is not None
    event_id = event_id_result[0] if event_id_result else None

# Print the event ID
    print("Event ID:", event_id)

# Consume any remaining results
    db.fetchall()
        # Inserting data into the timess table if the combination doesn't exist
    insert_query = """
            INSERT INTO times (swimmer_id, event_id, time, ts)
            SELECT %s, %s, %s, %s
            WHERE NOT EXISTS (
                SELECT * FROM times WHERE swimmer_id = %s AND event_id = %s AND time = %s
            );
        """
    for time in converts:
        db.execute(insert_query, (swimmer_id, event_id, swim_utils.build_time_string(time), dateTime, swimmer_id, event_id, swim_utils.build_time_string(time)))


Querying for swimmer ID with: Tasmin 15
Swimmer ID Result: (22,)
Swimmer ID: 22
Querying for event ID with: 200m Breast
Event ID Result: (11,)
Event ID: 11


In [16]:
# Fetch existing data from the 'swimmers' table
swimertable = []
swimertable_query = """SELECT name, age FROM swimmers"""
with DBcm.UseDatabase(config) as db:
    db.execute(swimertable_query)
    swimertable = db.fetchall()

# Extract existing names into a set for efficient membership checking
existing_names = [row for row in swimertable]

# Process swimmers_data1 to add new data if names don't exist in the database
swimmers_data1 = []
for filename in files:
    name = swim_utils.get_swimmers_data(filename)[0]
    age = swim_utils.get_swimmers_data(filename)[1]
    swimmers_data1.append((name,age))

SQL = """    
    INSERT INTO swimmers
    (name, age)
    VALUES
    (%s, %s)
"""

with DBcm.UseDatabase(config) as db:
    for (name, age) in swimmers_data1:
        if (name, age) not in existing_names:
            db.execute(SQL, (name, age))





In [17]:
existing_names

[('Abi', 10),
 ('Ali', 12),
 ('Alison', 14),
 ('Aurora', 13),
 ('Bill', 18),
 ('Blake', 15),
 ('Calvin', 9),
 ('Carl', 15),
 ('Chris', 17),
 ('Darius', 13),
 ('Dave', 17),
 ('Elba', 14),
 ('Emma', 13),
 ('Erika', 15),
 ('Hannah', 13),
 ('Katie', 9),
 ('Lizzie', 14),
 ('Maria', 9),
 ('Mike', 15),
 ('Owen', 15),
 ('Ruth', 13),
 ('Tasmin', 15),
 ('Abi', 10),
 ('Abi', 10),
 ('Abi', 10),
 ('Abi', 10),
 ('Abi', 10),
 ('Ali', 12),
 ('Ali', 12),
 ('Alison', 14),
 ('Alison', 14),
 ('Aurora', 13),
 ('Bill', 18),
 ('Bill', 18),
 ('Blake', 15),
 ('Blake', 15),
 ('Blake', 15),
 ('Calvin', 9),
 ('Calvin', 9),
 ('Calvin', 9),
 ('Carl', 15),
 ('Chris', 17),
 ('Chris', 17),
 ('Darius', 13),
 ('Darius', 13),
 ('Darius', 13),
 ('Darius', 13),
 ('Dave', 17),
 ('Dave', 17),
 ('Elba', 14),
 ('Emma', 13),
 ('Emma', 13),
 ('Erika', 15),
 ('Erika', 15),
 ('Erika', 15),
 ('Hannah', 13),
 ('Hannah', 13),
 ('Katie', 9),
 ('Katie', 9),
 ('Katie', 9),
 ('Katie', 9),
 ('Katie', 9),
 ('Katie', 9),
 ('Katie', 9),
 ('L

In [18]:
swimmers_data1

[('Abi', '10'),
 ('Abi', '10'),
 ('Abi', '10'),
 ('Abi', '10'),
 ('Abi', '10'),
 ('Ali', '12'),
 ('Ali', '12'),
 ('Alison', '14'),
 ('Alison', '14'),
 ('Aurora', '13'),
 ('Bill', '18'),
 ('Bill', '18'),
 ('Blake', '15'),
 ('Blake', '15'),
 ('Blake', '15'),
 ('Calvin', '9'),
 ('Calvin', '9'),
 ('Calvin', '9'),
 ('Carl', '15'),
 ('Chris', '17'),
 ('Chris', '17'),
 ('Darius', '13'),
 ('Darius', '13'),
 ('Darius', '13'),
 ('Darius', '13'),
 ('Dave', '17'),
 ('Dave', '17'),
 ('Elba', '14'),
 ('Emma', '13'),
 ('Emma', '13'),
 ('Erika', '15'),
 ('Erika', '15'),
 ('Erika', '15'),
 ('Hannah', '13'),
 ('Hannah', '13'),
 ('Katie', '9'),
 ('Katie', '9'),
 ('Katie', '9'),
 ('Katie', '9'),
 ('Katie', '9'),
 ('Katie', '9'),
 ('Katie', '9'),
 ('Lizzie', '14'),
 ('Lizzie', '14'),
 ('Maria', '9'),
 ('Mike', '15'),
 ('Mike', '15'),
 ('Mike', '15'),
 ('Mike', '15'),
 ('Mike', '15'),
 ('Owen', '15'),
 ('Ruth', '13'),
 ('Ruth', '13'),
 ('Ruth', '13'),
 ('Ruth', '13'),
 ('Ruth', '13'),
 ('Tasmin', '15'),
 ('

In [19]:
len(swimmers_data1)

60

In [20]:
import os
from statistics import mean

FOLDER = "swimdata/"

files = os.listdir(FOLDER)

files.remove('.DS_Store')

def convert2hundreths(timestring):
    """Given a string which represents a time, this function converts the string
    to a number (int) representing the string's hundredths of seconds value, which is
    returned.
    """
    if ":" in timestring:
        mins, rest = timestring.split(":")
        secs, hundredths = rest.split(".")
    else:
        mins = 0
        secs, hundredths = timestring.split(".")

    return int(hundredths) + (int(secs) * 100) + ((int(mins) * 60) * 100)

def build_time_string(num_time):
    """ """
    secs, hundredths = str(round(num_time / 100, 2)).split(".")
    mins = int(secs) // 60
    seconds = int(secs) - mins * 60
    return f"{mins}:{seconds}.{hundredths}"


def get_swimmers_data(filename):
    """ """
    name, age, distance, stroke = filename.removesuffix(".txt").split("-")
    with open(FOLDER + filename) as fh:
        data = fh.read()
    times = data.strip().split(",")
    converts = []  # empty list
    for t in times:
        converts.append(convert2hundreths(t))
    average = build_time_string(mean(converts))
    
    

    return name, age, distance, stroke, times, converts, average



In [21]:
swimmers_data2 = {}

for filename in files:
    name = get_swimmers_data(filename)[0]
    age = get_swimmers_data(filename)[1]
    swimmers_data2[name] = age
swimmers_data2

for name, age in swimmers_data2.items():
    print (name, "->", age )

Abi -> 10
Ali -> 12
Alison -> 14
Aurora -> 13
Bill -> 18
Blake -> 15
Calvin -> 9
Carl -> 15
Chris -> 17
Darius -> 13
Dave -> 17
Elba -> 14
Emma -> 13
Erika -> 15
Hannah -> 13
Katie -> 9
Lizzie -> 14
Maria -> 9
Mike -> 15
Owen -> 15
Ruth -> 13
Tasmin -> 15


In [22]:
len(swimmers_data2 )

22